In [156]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from sklearn.model_selection import train_test_split
url = 'https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv'

In [157]:
class Model(nn.Module):
    def __init__(self, inputs =4, hiddenlayer = 10, outputs = 3):
        super().__init__()
        self.layer1 = nn.Linear(inputs, hiddenlayer)
        self.output = nn.Linear(hiddenlayer, outputs)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = self.output(x)
        return x


torch.manual_seed(1)
model = Model()

df = pd.read_csv(url)

df['variety'] = df['variety'].replace(['Setosa', 'Versicolor', 'Virginica'], [0, 1, 2])

drop = df.drop('variety', axis=1)
accessor = df['variety']
drop = drop.values
accessor = accessor.values


drop_train, drop_test, accessor_train, accessor_test = train_test_split(drop, accessor, test_size=0.1, random_state=1)

drop_train = torch.FloatTensor(drop_train)
drop_test = torch.FloatTensor(drop_test)

accessor_train = torch.LongTensor(accessor_train)
accessor_test = torch.LongTensor(accessor_test)


errorMeasure = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

epochs = 1000
losses = []
for i in range(epochs):
        i = i+1
        y_pred = model.forward(drop_train)
        loss = errorMeasure(y_pred, accessor_train)
        losses.append(loss)

        if i % 100 == 0:
            print(f'epoch: {i} loss: {loss}')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()



<ipython-input-157-551909128c16>:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['variety'] = df['variety'].replace(['Setosa', 'Versicolor', 'Virginica'], [0, 1, 2])


epoch: 100 loss: 0.16345088183879852
epoch: 200 loss: 0.07769127935171127
epoch: 300 loss: 0.06301546841859818
epoch: 400 loss: 0.057583484798669815
epoch: 500 loss: 0.05462666228413582
epoch: 600 loss: 0.05254479497671127
epoch: 700 loss: 0.05086185038089752
epoch: 800 loss: 0.04942608252167702
epoch: 900 loss: 0.04819481819868088
epoch: 1000 loss: 0.04716175049543381


In [158]:
with torch.no_grad():
    y_eval = model.forward(drop_test)
    loss = errorMeasure(y_eval, accessor_test)
    print(loss)

tensor(0.0019)


In [159]:
correct = 0;

with torch.no_grad():

   for i, data in enumerate(drop_test):
       y_val = model.forward(data)

       if accessor_test[i] == 0:
           print('Setosa')
       elif accessor_test[i] == 1:
           print('Versicolor')
       else:
           print('Virginica')

       if y_val.argmax().item() == accessor_test[i]:
           correct += 1

       print(f'{y_val.argmax().item()}')

print(correct)

Setosa
0
Versicolor
1
Versicolor
1
Setosa
0
Virginica
2
Versicolor
1
Virginica
2
Setosa
0
Setosa
0
Virginica
2
Versicolor
1
Setosa
0
Virginica
2
Versicolor
1
Versicolor
1
15


In [160]:
newIris = torch.tensor([5.9, 3.0, 5.2, 1.8])

#user_input = input("Enter values")
#values = list(map(float, user_input.split(",")))
#newIris = torch.tensor(values)


with torch.no_grad():

    if(model(newIris).argmax().item() == 0):
        print('This flower is a Setosa')
    elif(model(newIris).argmax().item() == 1):
        print('This flower is a Versicolor')
    else:
        print('This flower is a Virginica')

This flower is a Virginica


In [161]:
torch.save(model.state_dict(), 'model.pt')

newModel = Model()
newModel.load_state_dict(torch.load('model.pt'))

newModel.eval()

<ipython-input-161-42bb093012f2>:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  newModel.load_state_dict(torch.load('model.pt'))


Model(
  (layer1): Linear(in_features=4, out_features=10, bias=True)
  (output): Linear(in_features=10, out_features=3, bias=True)
)